![LangChain](img/langchain.jpeg)

Les **Agents** dans LangChain ouvrent la voie à des systèmes plus dynamiques, capables de **raisonner étape par étape** et d’**interagir avec des outils** pour accomplir des tâches complexes.

Contrairement aux chaînes statiques (chains), **⚠️ un agent ne suit pas un chemin prédéfini**. **Il s’appuie sur un LLM** qui décide dynamiquement, à chaque étape, quelle action entreprendre : quel outil utiliser, quelles informations rechercher ou comment poursuivre, en fonction du contexte.

Les outils, ou **tools**, sont des fonctions encapsulées que l’agent peut appeler, il peut s'agir de fonctions pour interroger une base de données, de consulter une API, ou d’exécuter un calcul.

**Grâce à cette combinaison :**

> raisonnement du LLM → proposition d’action → exécution par l’agent → observation → nouveau raisonnement → et ainsi de suite...

... un agent LangChain devient un orchestrateur intelligent, capable de résoudre des problèmes ouverts ou de répondre à des requêtes complexes, sans suivre un script rigide.

![Agent](img/agent.png)

L’agent suit un **schéma itératif** basé sur le pattern **ReAct (Reasoning + Acting)**.  
À partir d’une requête, l'agent interagit avec un modèle de langage (LLM) qui raisonne étape par étape (**Thought**) et propose des actions (**Action**) à effectuer à l’aide d’outils disponibles.   
L’agent exécute ces actions, collecte les résultats (**Observation**), et les renvoie au LLM pour affiner son raisonnement.   
Ce cycle **ReAct** se répète jusqu’à ce que le LLM formule une réponse finale (**Final Answer**), que l’agent retourne à l’utilisateur.

![Hugging Face](img/hugging_face.jpeg)

# 1. Chargement du modèle LLM local
___

Dans cette section, nous chargeons un modèle de langage local grâce à **Ollama**. Cela permet de travailler avec un **LLM directement sur notre machine**, sans connexion à une API externe.

Nous utilisons ici la classe `ChatOllama` de **LangChain**, qui nous permet d’interagir facilement avec un modèle comme **llama3** déjà téléchargé via Ollama.

In [1]:
import os
from IPython.display import display, clear_output, Markdown
from dotenv import load_dotenv
from datetime import datetime
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from langchain import hub
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory

# Chargement des clés d'API se trouvant dans le fichier .env.  
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

model = ChatOllama(model="llama3", temperature=0)
#model = ChatDeepSeek(model="deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))

# 2. Agent standard
___

Un agent standard permet d’utiliser un modèle de langage avec un ou plusieurs **outils externes**, comme des fonctions Python, pour répondre à une tâche spécifique.  
Cet agent **fonctionne sans mémoire** : il ne conserve **aucun historique des interactions précédentes**. Chaque question est traitée de manière indépendante, comme une **requête isolée**.

Dans cet exemple, l’agent utilise un outil simple pour répondre à la question « Quelle heure est-il ? », en appelant une fonction qui retourne l’heure actuelle.
Son comportement est guidé par un prompt ReAct standard chargé depuis LangChain Hub, qui lui permet de raisonner et de décider quand utiliser un outil.

### 2.1 Préparation des outils

In [2]:
# Définition de l'outil : retourne l'heure actuelle au format HH:MM
def get_current_time(*args, **kwargs):
    return datetime.now().strftime("%H:%M")

# Liste des outils que l'agent peut utiliser. Chaque outil est exposé au LLM via un nom et une description.
# Cela permet au LLM, durant son raisonnement, de décider quel outil appeler en fonction de la tâche à accomplir.
# Ici, un seul outil est défini : "CurrentTime", qui retourne l'heure actuelle au format HH:MM.
tools = [
    Tool(
        name="CurrentTime",
        func=get_current_time,
        description="Use this tool to get the current time."
    )
]

### 2.2 Préparation et usage de l'agent

In [3]:
# Chargement du prompt standard pour le paradigme ReAct depuis LangChain Hub
prompt = hub.pull("hwchase17/react")

# Création de l'agent ReAct avec le modèle, les outils et le prompt
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Encapsulation de l’agent dans un exécuteur avec configuration de contrôle
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True    # Affiche les étapes de raisonnement (utile en debug)
)

# Lancement de l’agent avec une question en entrée
response = executor.invoke({"input": "Quelle heure est-il ?"})

display(Markdown(response["output"]))

c:\Users\User\Desktop\LANGCHAIN_agent_tools\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Let's get started!

Thought: Since the question is asking for the current time, I should use the CurrentTime tool.

Action: CurrentTime
Action Input: None (since we don't need any additional inputs)13:57Here are my answers:

Question: Quelle heure est-il ?
Thought: Let's get started!

Thought: Since the question is asking for the current time, I should use the CurrentTime tool.

Action: CurrentTime
Action Input: None (since we don't need any additional inputs)13:58

KeyboardInterrupt: 

### 🧩 Exercices

> Exercice 1

Créez un agent capable de faire des conversions de température. Votre agent doit pouvoir répondre à des questions comme :
- *“Quelle est la température en Celsius pour 100 Fahrenheit ?”*
- *“Convertis 37.5 degrés Celsius en Fahrenheit.”*

💡 Utilisez 2 **tools** différents

💪🏻 Bonus : Autoriser des entrées plus souples, comme “Convertis 100 F en C” ou “Celsius pour 212°F”.

In [6]:
# Définition des outils
def celcius_to_fahrenheit(temperature_c):
    return round((float(temperature_c)  * (9/5)) + 32, 2)
def fahrenheit_to_celcius(temperature_f):
    return round((float(temperature_f)  - 32) *(5/9), 2)

# Liste des outils 
tools = [
    Tool(
        name="TemperatureF",
        func=celcius_to_fahrenheit,
        description="Use this tool to convert temperature from celcius to fahrenheit."
    ), Tool(
        name="TemperatureC",
        func=fahrenheit_to_celcius,
        description="Use this tool to convert temperature from fahrenheit to celcius."
    )
]

In [8]:
# Chargement du prompt standard pour le paradigme ReAct depuis LangChain Hub
prompt = hub.pull("hwchase17/react")

# Création de l'agent ReAct avec le modèle, les outils et le prompt
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Encapsulation de l’agent dans un exécuteur avec configuration de contrôle
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True    # Affiche les étapes de raisonnement (utile en debug)
)

# Lancement de l’agent avec une question en entrée
response = executor.invoke({"input": "Convertis 100 F en C ?"})

display(Markdown(response["output"]))

c:\Users\User\Desktop\LANGCHAIN_agent_tools\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: To convert 100 F to C, I need to use the TemperatureF tool.

Action: TemperatureC
Action Input: 10037.78Here's my attempt at answering the question:

Question: Convertis 100 F en C ?

Thought: To convert 100 F to C, I need to use the TemperatureF tool.

Action: TemperatureC
Action Input: 10037.78I think I have it!

Thought: The answer is already given in the observation.
Final Answer: 37.78°C

> Finished chain.


37.78°C

# 2. Agent conversationnel
___

Un agent conversationnel est conçu pour gérer un **dialogue continu**, en conservant une **mémoire des échanges précédents**. Contrairement à l’agent standard qui traite chaque requête indépendamment, un agent conversationnel **peut adapter ses réponses en fonction du contexte accumulé dans la conversation**.

Ce type d’agent est particulièrement utile pour construire des assistants interactifs, des conseillers ou des systèmes de FAQ qui doivent s’adapter aux intentions de l’utilisateur au fil du temps.

LangChain permet d’ajouter une mémoire à un agent grâce à des modules comme `ConversationBufferMemory`, afin que le modèle de langage puisse prendre en compte l’historique des échanges lors de chaque nouvelle interaction.

In [ ]:
# Récupération d’un prompt conversationnel React (basé sur ReAct)
prompt = hub.pull("hwchase17/react-chat")

# Initialisation de la mémoire pour suivre l’historique des échanges
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Création de l'agent ReAct avec le modèle, les outils et le prompt
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Construction d’un exécuteur qui lie l’agent, les outils et la mémoire
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True    # Affiche les étapes de raisonnement (utile en debug)
)

# Boucle interactive terminale
while True:
    user_input = input("Vous : ")
    clear_output(wait=True)                         # Efface l'affichage précédent
    display(Markdown(f"**Vous :** {user_input}"))   # Affiche la requête de l'utilisateur

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    response = executor.invoke({"input": user_input})
    display(Markdown(response["output"]))

### 🧩 Exercices

> Exercice 1

Reprenez vos travaux sur l'exercice précédent pour y introduire un aspect conversationnel grâce à une boucle de conversation et à la gestion de la mémoire (`ConversationBufferMemory`).

In [ ]:
# Définition des outils
def celcius_to_fahrenheit(temperature_c):
    return round((float(temperature_c)  * (9/5)) + 32, 2)
def fahrenheit_to_celcius(temperature_f):
    return round((float(temperature_f)  - 32) *(5/9), 2)

# Liste des outils 
tools = [
    Tool(
        name="TemperatureF",
        func=celcius_to_fahrenheit,
        description="Use this tool to convert temperature from celcius to fahrenheit."
    ), Tool(
        name="TemperatureC",
        func=fahrenheit_to_celcius,
        description="Use this tool to convert temperature from fahrenheit to celcius."
    )
]

In [ ]:
# Récupération d’un prompt conversationnel React (basé sur ReAct)
prompt = hub.pull("hwchase17/react-chat")

# Initialisation de la mémoire pour suivre l’historique des échanges
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Création de l'agent ReAct avec le modèle, les outils et le prompt
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Construction d’un exécuteur qui lie l’agent, les outils et la mémoire
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True    # Affiche les étapes de raisonnement (utile en debug)
)

# Boucle interactive terminale
while True:
    user_input = input("Vous : ")
    clear_output(wait=True)                         # Efface l'affichage précédent
    display(Markdown(f"**Vous :** {user_input}"))   # Affiche la requête de l'utilisateur

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    response = executor.invoke({"input": user_input})
    display(Markdown(response["output"]))